In [1]:
import pprint

""" Import the GreenTranslator API wrapper. """
from greentranslator.api import GreenTranslator

""" Create an instance. """
translator = GreenTranslator ()

In [2]:
""" Query the exposure service. TODO: exercise other exposure endpoints. """
exposure = translator. \
    get_exposures(). \
    get_exposure_by_area (exposure_type = 'pm25',
                        latitude      = '',
                        longitude     = '',
                        radius        = '0')
pprint.pprint (exposure, indent=2)

[ {'latitude': '35.7795897', 'longitude': '-78.6381787'},
  {'latitude': '35.9131996', 'longitude': '-79.0558445'},
  {'latitude': '35.9940329', 'longitude': '-78.898619'}]


In [3]:
""" 
3. What pediatric medications are available that act on the protein (gene) targets and biological pathways that are activated or inhibited by PM2.5 and ozone exposure?
  - This query will be extended to be a building block for this query.
  - Get drugs by condition. 
"""
drugs = translator.\
    get_medbiochem().\
    get_drugs_by_condition (conditions=[ "d001249" ])
pprint.pprint (drugs[:10], indent=2)

[ 'Melphalan',
  'Saquinavir',
  'Gefitinib',
  'Cyclophosphamide',
  'Fluorouracil',
  'Furosemide',
  'Doxorubicin',
  'Genistein',
  'Enflurane',
  'Melatonin']


In [6]:
""" 
2. What protein (gene) targets and biological pathways are activated or inhibited by PM2.5 and ozone exposure?
   - This note is a building block for the second component of this query.
   - Get genes and pathways assoicated with a set of conditions. 
"""
conditions = [ 'd001249', 'd003371', 'd001249' ]
genes_paths = translator.\
    get_medbiochem().\
    get_genes_pathways_by_disease (diseases = conditions)
pprint.pprint (genes_paths[:3], indent=2)

[ { 'human  ': True,
    'keggPath': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa00010',
    'pathName': 'path:hsa00010 Glycolysis / Gluconeogenesis - Homo sapiens '
                '(human)',
    'uniprotGene': 'http://chem2bio2rdf.org/uniprot/resource/gene/ALDH2'},
  { 'human  ': True,
    'keggPath': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa00010',
    'pathName': 'path:hsa00010 Glycolysis / Gluconeogenesis - Homo sapiens '
                '(human)',
    'uniprotGene': 'http://chem2bio2rdf.org/uniprot/resource/gene/ALDH2'},
  { 'human  ': True,
    'keggPath': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa00010',
    'pathName': 'path:hsa00010 Glycolysis / Gluconeogenesis - Homo sapiens '
                '(human)',
    'uniprotGene': 'http://chem2bio2rdf.org/uniprot/resource/gene/ALDH2'}]


In [5]:
""" Get genes and pathways involved in exposure to a chemical. We'll use this to determine the 
    effects of PM2.5, or, more generally, of 'Particulate Matter', the MeSH term. """ 
exposures = translator.\
    get_medbiochem().\
    get_exposure_conditions (chemicals = [ 'D052638' ])
pprint.pprint (exposures[:5], indent=2)

[ { 'chemical': 'http://bio2rdf.org/mesh:D052638',
    'gene': 'http://chem2bio2rdf.org/uniprot/resource/gene/MAPK1',
    'human': True,
    'pathID': 'hsa04664',
    'pathName': 'path:hsa04664 Fc epsilon RI signaling pathway - Homo sapiens '
                '(human)',
    'pathway': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa04664'},
  { 'chemical': 'http://bio2rdf.org/mesh:D052638',
    'gene': 'http://chem2bio2rdf.org/uniprot/resource/gene/MAPK1',
    'human': True,
    'pathID': 'hsa04664',
    'pathName': 'path:hsa04664 Fc epsilon RI signaling pathway - Homo sapiens '
                '(human)',
    'pathway': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa04664'},
  { 'chemical': 'http://bio2rdf.org/mesh:D052638',
    'gene': 'http://chem2bio2rdf.org/uniprot/resource/gene/MAPK1',
    'human': True,
    'pathID': 'hsa04664',
    'pathName': 'path:hsa04664 Fc epsilon RI signaling pathway - Homo sapiens '
                '(human)',
    'pathway': 'http://chem2bio2

In [8]:
print (open ('greentranslator/query/ctd_gene_expo_disease.sparql').read ())

prefix chemgene: <http://chem2bio2rdf.org/ctd/resource/ctd_chem_gene/>
prefix mesh:     <http://bio2rdf.org/mesh:>
prefix ctd:      <http://chem2bio2rdf.org/ctd/resource/>
prefix kegg:     <http://chem2bio2rdf.org/kegg/resource/>
prefix pharmgkb: <http://chem2bio2rdf.org/pharmgkb/resource/>

select ?chemical ?gene ?kegg_pathway ?pathway_name ?pathway_id where {

   values ( ?chemical ) {
      $chemicals
   }

   ?chemgene      ctd:chemicalid       ?chemical ;
                  ctd:interaction      ?interaction ;
                  ctd:interactiontypes ?interaction_types;
                  ctd:gene             ?gene .
 
   ?pharmgkb_gene pharmgkb:Symbol      ?gene ;
                  pharmgkb:UniProt_Id  ?uniprotid .
   
   ?kegg_pathway  kegg:protein         ?uniprotid ;
                  kegg:Pathway_name    ?pathway_name ;
                  kegg:description     ?pathway_descr ;
                  kegg:pathwayid       ?pathway_id ;
                  kegg:ID              ?pathway_nid .


In [9]:
print (open('greentranslator/query/genes_pathways_by_disease.sparql').read ())

prefix kegg:      <http://chem2bio2rdf.org/kegg/resource/>
prefix pharmgkb:  <http://chem2bio2rdf.org/pharmgkb/resource/>
prefix drugbank:  <http://chem2bio2rdf.org/drugbank/resource/>
prefix pubchem:   <http://chem2bio2rdf.org/pubchem/resource/>
prefix ctd:       <http://chem2bio2rdf.org/ctd/resource/>
prefix omim:      <http://chem2bio2rdf.org/omim/resource/>
prefix mesh:      <http://bio2rdf.org/mesh:>

select ?diseaseID ?drugGenericName ?swissProtID ?uniprotGeneID ?geneBankID ?pathwayName ?keggPath where {

    ?keggPath    kegg:protein                ?swissProtID ;
                 kegg:Pathway_name           ?pathwayName .
    ?keggInter   kegg:cid                    ?pubchemCID .

    ?dbInter     drugbank:GeneBank_ID        ?geneBankID ;
                 drugbank:SwissProt_ID       ?swissProtID ;
                 drugbank:gene               ?uniprotGeneID ;
                 drugbank:Name               ?name ;
                 drugbank:DBID               ?drugID .
  
    ?drugID